In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from utils import HelperUtils, Configuration

In [2]:
initial_data_dir = HelperUtils.load_data('./data/simple.zip', "initial_dataset")
HelperUtils.convert_dataset(initial_data_dir)

1741168640/1741154793 [==============================] - 2s 0us/step


In [3]:
# This only assigns labels that mean what callection brick belongs to. It does no assign exact brick number
def prepare_datasets(data_dir):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="training",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="validation",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    class_names = train_ds.class_names

    num_classes = len(class_names)

    # train_ds = train_ds.shuffle(500).batch(batch_size)
    # val_ds = val_ds.shuffle(500).batch(batch_size)
    return train_ds, val_ds, num_classes, class_names

In [4]:
initial_train_ds, initial_val_ds, initial_num_classes, class_names = prepare_datasets(initial_data_dir)

Found 77638 files belonging to 433 classes.
Using 62111 files for training.
Found 77638 files belonging to 433 classes.
Using 15527 files for validation.


In [5]:
model = Sequential([
    layers.Rescaling(1. / 255, input_shape=(Configuration.img_height, Configuration.img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(initial_num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

In [ ]:
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# %load_ext tensorboard
# %tensorboard --logdir logs

history = model.fit(
    initial_train_ds,
    validation_data=initial_val_ds,
    epochs=Configuration.epochs,
    # callbacks=[tensorboard_callback]
)

model.save(f'{Configuration.models_location}/initial_model.h5')

Epoch 1/10
1941/1941 [==============================] - 66s 32ms/step - loss: 3.1772 - accuracy: 0.3285 - val_loss: 1.5258 - val_accuracy: 0.6054
Epoch 2/10
1941/1941 [==============================] - 38s 20ms/step - loss: 1.0399 - accuracy: 0.7178 - val_loss: 0.9539 - val_accuracy: 0.7443
Epoch 3/10
1941/1941 [==============================] - 39s 20ms/step - loss: 0.5048 - accuracy: 0.8547 - val_loss: 0.7814 - val_accuracy: 0.8059
Epoch 4/10
1941/1941 [==============================] - 38s 19ms/step - loss: 0.3008 - accuracy: 0.9085 - val_loss: 0.7982 - val_accuracy: 0.8188
Epoch 5/10
1941/1941 [==============================] - 38s 20ms/step - loss: 0.2084 - accuracy: 0.9352 - val_loss: 0.8271 - val_accuracy: 0.8277
Epoch 6/10
1939/1941 [============================>.] - ETA: 0s - loss: 0.1705 - accuracy: 0.9475

In [ ]:
HelperUtils.test(model, class_names)